# urllib - xpath - 自由時報關鍵字,內文摘要 - 輸出JSON檔案

In [ ]:
#coding:utf-8
#65001
import urllib.request
import json
import codecs
import sys
import argparse as ap
import time
import datetime
import lxml.html
from urllib.parse import quote
from random import randint

#python main.py 世大運 2015-07-01 2017-07-03 1
# def argParse():
#     parser=ap.ArgumentParser(description='Liberty Time Net Crawler')
#     parser.add_argument("keyword", help="Serch Keyword")
#     parser.add_argument("start_date", help="Start (2017-01-01)")
#     parser.add_argument("end_date", help="End (2017-01-02)")
#     parser.add_argument("pages", help="Pages")
#     return parser.parse_args()

# args=argParse()
# keyword = quote(args.keyword)
# start_date = args.start_date
# end_date = args.end_date
# pages = args.pages

keyword = quote('肺炎')
start_date = '2020-04-14'
end_date = '2020-04-15'
pages = '1'


def start_requests():
    start_list = start_date.split("-")
    end_list = end_date.split("-")
    SYear = ''
    SMonth = ''
    SDay = ''
    EYear = ''
    EMonth = ''
    EDay = ''
    if(len(start_list)==3) and (len(end_list)==3):
        SYear = start_list[0]
        SMonth = start_list[1]
        SDay = start_list[2]
        EYear = end_list[0]
        EMonth = end_list[1]
        EDay = end_list[2]
    else:
        print ("Date format error.")
  
    urls = []
    for i in range(1,int(pages)+1):
        str_idx = ''+('%s' % i)
        #http://news.ltn.com.tw/search?keyword=世大運&conditions=and&SYear=2015&SMonth=6&SDay=27&EYear=2015&EMonth=8&EDay=24&page=1
        api = 'http://news.ltn.com.tw/search?keyword='+keyword+'&conditions=and&SYear='+SYear+'&SMonth='+SMonth+'&SDay='+SDay+'&EYear='+EYear+'&EMonth='+EMonth+'&EDay='+EDay+'&page='+str_idx+''
        api = 'https://news.ltn.com.tw/search?keyword='+keyword+'&conditions=and&start_time='+start_date+'&end_time='+end_date+'&page='+str_idx
        urls.append(api)
        for url in urls:
            #print (url)
            parseLtnNews(url)
            time.sleep(0.2)


def parseLtnNews(uri):
    print(uri)
    handle = urllib.request.urlopen(uri)
    encoding = handle.headers.get_content_charset()
    html_data =  handle.read().decode(encoding)
    selector = lxml.html.document_fromstring(html_data)
    newslist = selector.xpath('//*[@class="searchlist boxTitle"]/li')
    print(range(len(newslist)))
    for i in range(len(newslist)):
        strTitle = ''
        strUrl = ''
        strBody = ''
        strDate = ''
        str_idx = str(i+1)
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/a//text()'
        titleList = selector.xpath(str_xpath)
        strTitle = " ".join(titleList)
        print(strTitle)
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/a//@href'
        urlList = selector.xpath(str_xpath)[0]
        strUrl = ''.join(urlList)
        strUrl = strUrl
        print(strUrl)
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/p//text()'
        bodyList = selector.xpath(str_xpath)
        strBody = ''.join(bodyList).replace('\n','')
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/span//text()'
        dateList = selector.xpath(str_xpath)
        strDate = ''.join(dateList).replace("&nbsp;","")[:10]
        if len(strTitle)>1:
            items.append({
                "title": strTitle,
                "link":strUrl,
                "body":strBody,
                "postdate":strDate,
                #"updatetime":datetime.datetime.now(),  # MongoDB
                "updatetime":datetime.datetime.now().strftime('%Y-%m-%d')
                })
    handle.close()



if __name__ == '__main__':
    items = []
    start_requests()
    row_json = json.dumps(items, ensure_ascii=False)
    file = codecs.open(urllib.parse.unquote(keyword)+'.json', 'w', encoding='utf-8')
    #file = codecs.open('out.json', 'w', encoding='utf-8')
    file.write(row_json)
    file.close()

    print("Done")